In [6]:
import numpy as np
import string

In [8]:
N = 26  # number of stocks
n = 10  # top 10 stocks
tickers = np.array(list(string.ascii_uppercase))
years = 20
prices = np.random.random((1,N))*10 + 100
indicators = np.random.random((1,N))*100
volumes = np.random.random((1,N))*1000000 + 10000000

In [9]:
for i in range(12*years-1):
    prices = np.vstack((prices, prices[-1] * (1 + (np.random.random(N)-0.5)/5))) # multiply by 1+[-0.1,0.1]
    indicators = np.vstack((indicators, indicators[-1] + 6*np.random.random(N)-3)) # add [-3, 3] 
    volumes = np.vstack((volumes, volumes[-1] * (1 + (np.random.random(N)-0.5)/5))) # multiply by 1+[-0.1,0.1]

In [104]:
class algo(object):
    def __init__(self, initial_prices, initial_indicators, initial_volume): 
                     # intial_prices, initial_indicators, initial_volume are N-d arrays
        self.months = 0
        self.avail_capital = 100000000
        
        self.price_arr = initial_prices
        self.indicator_arr = initial_indicators
        self.volume_arr 
        # Initialize portfolio
        self.top_indices = np.argsort(-self.indicator_arr)[:n] # argsort is ascending so negate indicator_arr
        
        self.entry_months = np.array([-1]*N) # -1 indicates there is no position so no entry month
        self.entry_months[self.top_indices] = 0
        
        self.shares = np.array([0]*N)
        self.shares[self.top_indices] = (self.avail_capital/n/self.price_arr[self.top_indices]).astype(int)
        self.avail_capital = self.avail_capital - np.sum(self.price_arr*self.shares)        
        
    def process_new_data(self, new_prices, new_indicators, new_volumes):
        self.months += 1
        self.price_arr = np.vstack((self.price_arr, new_prices))
        self.indicator_arr = np.vstack((self.indicator_arr, new_indicators))
        self.volume_arr = np.vstack((self.volume_arr, new_volumes))
        
        self.update_portfolio()
        
    def update_portfolio(self):
        self.new_top_indices = np.argsort(-self.indicator_arr[-1])[:n]
        sell_indices = []
        buy_indices = []
        for x in self.top_indices:# Try to add new indices not currently in portfolio to buy_indices
            if (self.months-self.entry_months[x] >= 12) and (x not in new_top_indices): # if held for >=1yr and not in new top indices
                sell_indices += [x]
                
        for y in self.new_top_indices: # Add k best new indices to portfolio, k = number of stocks we will sell
            if y not in self.top_indices:
                buy_indices += [y]
                if len(buy_indices) == len(sell_indices): # Stop once we have same number of buy as sell
                    break
        
        self.execute_sell(sell_indices)
        self.execute_buy(buy_indices)
    
    def execute_sell(self, sell_indices):
        print('selling', tickers[sell_indices])
        for i in sell_indices:
            sell_gain = self.price_arr[-1, i] *self.shares[i]
            transaction_cost = 0.01 * self.shares[i]
            slip_cost = 0
            if self.shares[i] >= 0.1 * self.volume_arr[-1, i]:
                slip_cost = self.price_arr[-1, i] * 0.01 * self.shares[i]
            
            self.avail_capital += sell_gain - transaction_cost - slip_cost # Update capital
            self.shares[i] = 0 # Reset shares
            self.entry_months[i] = -1 # Reset entry_mont to -1 to denote no position
            
        
    def execute_buy(self, buy_indices):
        print('buying', tickers[buy_indices])
        for i in buy_indices:
            buy_money = self.avail_capital/len(buy_indices) # Allocate avail_capital equally 
            buy_price = self.price_arr[-1, i] + 0.01
            n_shares = int(buy_money/buy_price)
            if n_shares >= 0.1 * self.volume_arr[-1, i]:
                buy_price = self.price_arr[-1, i] + 0.01 + 0.1*self.price_arr[-1, i]
                n_shares = int(buy_money/buy_price)
                
            self.shares[i] = n_shares
            self.entry_months[i] = self.months
            self.avail_capital = self.avail_capital - n_shares*buy_price
        

In [42]:
#N = 100
#a = algo(p0, i0, v0)
#for i in range(1, N):
    #a.process_new_data(pi, ii, vi)
    
import pandas as pd
df = pd.read_csv('data.dat', sep=' ')

df['year'] = df['year'].map(lambda x: str(x))
df['month'] = df['month'].map(lambda x: str(x))
df['-'] = '-'
df['timestemp'] = df['year'] + df['-'] + df['month']
df['timestemp'] = pd.to_datetime(df['timestemp'])
df = df.set_index('timestemp')
df = df.drop(['active', 'year', 'month', '-', 'date'], axis=1)
df['volume_monthly'] = df['csho_1yr_avg']/4
df['EBIT'] = df['saleq_ttm'] - df['cogsq_ttm'] - df['xsgaq_ttm']
df['EBIT/EV'] = df['EBIT'] / df['entval']

#df.columns
#df1 = df[['timestemp','gvkey','adjusted_price','EBIT/EV'], axis=1]
df.drop(['gics-sector', 'mom1m', 'mom3m', 'mom6m', 'mom9m', 'mrkcap',
       'entval', 'saleq_ttm', 'cogsq_ttm', 'xsgaq_ttm', 'oiadpq_ttm',
       'niq_ttm', 'cheq_mrq', 'rectq_mrq', 'invtq_mrq', 'acoq_mrq',
       'ppentq_mrq', 'aoq_mrq', 'dlcq_mrq', 'apq_mrq', 'txpq_mrq', 'lcoq_mrq',
       'ltq_mrq', 'seqq_mrq', 'atq_mrq', 'csho_1yr_avg',
       'prccm', 'ajexm', 'EBIT',], axis=1, inplace=True)

df['indicator'] = df['EBIT/EV'].shift(periods = -12)
df.head()    
## use data from 2000 to 2017

,gvkey,adjusted_price,volume_monthly,EBIT/EV,indicator
timestemp,,,,,
1994-11-01,61527,0.0,0.0,NaN,-inf
1994-12-01,61527,0.0,0.0,NaN,-0.614144
1995-01-01,61527,0.0,0.0,-inf,-0.834988
1995-02-01,61527,0.0,0.0,-inf,-0.024139
1995-03-01,61527,0.0,0.0,-inf,-0.020596


In [43]:
df.drop(['EBIT/EV'],axis=1, inplace=True)
df.head()

,gvkey,adjusted_price,volume_monthly,indicator
timestemp,,,,
1994-11-01,61527,0.0,0.0,-inf
1994-12-01,61527,0.0,0.0,-0.614144
1995-01-01,61527,0.0,0.0,-0.834988
1995-02-01,61527,0.0,0.0,-0.024139
1995-03-01,61527,0.0,0.0,-0.020596


In [46]:
df2 = df.pivot(columns = 'gvkey', values = ['adjusted_price','indicator','volume_monthly'])
df2.head()

adjusted_price                                                      \
gvkey              1004   1013   1021   1034     1045   1056   1072    1075     
timestemp                                                                       
1970-05-01            NaN    NaN    NaN    NaN      NaN    NaN    NaN     NaN   
1970-06-01       1.424729    NaN    NaN    NaN      NaN    NaN    NaN     NaN   
1970-07-01       1.427530    NaN    NaN    NaN   8.1250    NaN    NaN  17.125   
1970-08-01       1.476755    NaN    NaN    NaN   9.4375    NaN    NaN  18.375   
1970-09-01       1.985415    NaN    NaN    NaN  10.8750    NaN    NaN  18.500   

                             ...   volume_monthly                              \
gvkey      1076      1078    ...           295786 296318 296944 297209 311524   
timestemp                    ...                                                
1970-05-01    NaN       NaN  ...              NaN    NaN    NaN    NaN    NaN   
1970-06-01    NaN       NaN  ...              NaN    NaN    NaN    NaN    NaN   
1970-07-01    NaN  0.480469  ...              NaN    NaN    NaN    NaN    NaN   
1970-08-01    NaN  0.494141  ...              NaN    NaN    NaN    NaN    NaN   
1970-09-01    NaN  0.487305  ...              NaN    NaN    NaN    NaN    NaN   

                                               
gvkey      312009 314866 315318 316056 317427  
timestemp                                      
1970-05-01    NaN    NaN    NaN    NaN    NaN  
1970-06-01    NaN    NaN    NaN    NaN    NaN  
1970-07-01    NaN    NaN    NaN    NaN    NaN  
1970-08-01    NaN    NaN    NaN    NaN    NaN  
1970-09-01    NaN    NaN    NaN    NaN    NaN  

[5 rows x 11370 columns]

In [74]:
df3 = df2.copy()

In [75]:
df3  = df3.reset_index()

In [95]:
A = {}
for i in range(len(df3)):
    A[i] = {}
    rr = np.array(df3.loc[0][1:]).tolist()
    A[i]['price'] = rr[:3791]
    A[i]['indicator'] = rr[3791:7581]
    A[i]['volume'] = rr[7581:]

In [102]:
initial_prices, initial_indicators, initial_volume = A[0]['price'], A[0]['indicator'],A[0]['volume']